In [2]:
# Download anexos
#
#alterar filtro fornecedores do arquivo  
#ALTERAR NOME ARQUIVO EXCEL    
#ALTERAR TITULO DO E-MAIL 
#ALTERAR E-MAIL            
#
#
import datetime
import os
import win32com.client
from pywintypes import com_error

path = os.path.expanduser(r"C:\Users\matheus.arruda\Desktop\Matheus")
today = datetime.date.today()

outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

try:
    inbox2=outlook.Folders.Item(1).Folders['Caixa de Entrada'].Folders['DOWNLOADS']
    
except(com_error): 
    inbox2=outlook.Folders.Item(2).Folders['Caixa de Entrada'].Folders['DOWNLOADS']

#item 2 = outlook fechado

messages = inbox2.Items

#for message in messages:
 #   print(message.Subject)
    

def salva_anexo(subject):
    for message in messages:
        if message.Subject == subject and message.Senton.date() == today:
            # body_content = message.body
            attachments = message.Attachments
            attachment = attachments.Item(1)
            for attachment in message.Attachments:
                attachment.SaveAsFile(os.path.join(path, str(attachment)))
                if message.Subject == subject and message.Unread:
                    message.Unread = False
                break

                
assunto = ('Open PO - FUP - StdWork')                
salva_anexo(assunto)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

vendas_df = pd.read_excel("Open PO - V5.xlsx")

#display(vendas_df.head(10))
print(vendas_df.shape)

#FILTRO FORNECEDORES (USANDO ISIN, MELHOR PARA FILTROS COM LISTA) 
#https://www.youtube.com/watch?v=E7e74mqk54o&list=PL4OAe-tL47sa1McMctk5pdPd5eTAp3drk&index=12
vendas_df.rename(columns={'Vendor Name' : 'Vendor_Name'}, inplace = True) # tirando o espaço do cabeçalho p usar inpalce filtro
filtro = vendas_df.isin({"Vendor_Name" : ["
                                          #NOME DE UM FORNECEDOR DA CARTEIRA
                                          "]})
vendas_df=vendas_df[(filtro.Vendor_Name)]
print(vendas_df.shape)

#formatos DATAS TENT
vendas_df['PO Creation Date'] = pd.to_datetime(vendas_df["PO Creation Date"],format = '%d/%b/%y').dt.date
vendas_df['Need By Date - Original'] = pd.to_datetime(vendas_df["Need By Date - Original"],format = '%d/%b/%y').dt.date
vendas_df['Need By Date - Atual'] = pd.to_datetime(vendas_df["Need By Date - Atual"],format = '%d/%b/%y').dt.date
vendas_df['Promise Date'] = pd.to_datetime(vendas_df["Promise Date"],format = '%d/%b/%y').dt.date
vendas_df['New Due Date (Demand)'] = pd.to_datetime(vendas_df["New Due Date (Demand)"],format = '%d/%b/%y').dt.date
vendas_df['Previsao Pegging'] = pd.to_datetime(vendas_df["Previsao Pegging"],format = '%d/%b/%y').dt.date
vendas_df['New Need By Date (PO)'] = pd.to_datetime(vendas_df["New Need By Date (PO)"],format = '%d/%b/%y').dt.date

#colunas com datas corretas
from pandas.tseries.offsets import BDay

def dias_uteis(from_date, add_days):
    business_days_to_add = add_days
    current_date = from_date
    
    current_date = current_date - BDay(business_days_to_add)
    return current_date


vendas_df[['Acceptable Early Days']] = vendas_df[['Acceptable Early Days']].fillna(0)
vendas_df['Acceptable Early Days'] = vendas_df['Acceptable Early Days'].astype(np.int64)
vendas_df[['Receiving Status']] = vendas_df[['Receiving Status']].fillna(0)
vendas_df['Faturar a partir de'] = vendas_df.apply(lambda x: dias_uteis(x['Need By Date - Atual'], x['Acceptable Early Days']), axis=1)
vendas_df['Faturar a partir de'] = vendas_df['Faturar a partir de'] - timedelta(days=1)
vendas_df['Faturar a partir de'] = pd.to_datetime(vendas_df["Faturar a partir de"],format = '%d-%b-%y').dt.date

#imputando colunas
vendas_df["Ação"]=''
vendas_df["Obs"] = ''
vendas_df["Status2"] = ''
vendas_df["Custom"] = ''
vendas_df["WF Frete"] = ''

#FILTRO recebidos
vendas_df.rename(columns={'Receiving Status' : 'Receiving_Status'}, inplace = True) # tirando o espaço do cabeçalho p usar inpalce filtro
filt = vendas_df.isin({"Receiving_Status" : [0,"Partial Receipt"]})
vendas_df=vendas_df[(filt.Receiving_Status)]
print(vendas_df.shape)

#FILTRO COLUNAS 
#vendas_df = vendas_df.loc[:,["Status Auto x Manual","Warehouse Code","Custom","Comments","PO Number","Vendor Name"]]
vendas_df = vendas_df.loc[:,["Warehouse Code","PO Number","Line Number","Vendor_Name","Vendor Site Code","Product Number","Product Description","Unit Of Measure","Qty Ordered","Qtd Pendente","Unit Price","Extended Price","Need By Date - Atual","New Need By Date (PO)","Promise Date","Promise Status","Ação","Faturar a partir de","PO Creation Date","Need By Date - Original","Cum LT Dias Corridos","Acceptable Early Days","Nota Fiscal","Receiving_Status","Receiving Qty","ShortCycle? PO NeedByDate"]]

#FAZENDO REGRA PROMISE

date_before = datetime(2018,1,19)

date_before = date_before.date()

vendas_df.loc[vendas_df['Promise Date'] < date_before, 'Promise Date'] = ''

#fazendo regras de ação p/ fornecedor
vendas_df.loc[vendas_df['Faturar a partir de'] <= today, 'Ação']= 'Faturar pedido'  
vendas_df.loc[vendas_df['Faturar a partir de'] > today, 'Ação']= 'Aguardar liberação, não autorizado'
vendas_df.loc[(vendas_df['Faturar a partir de'] > today) & (vendas_df['Promise Date'] == ''), 'Ação']= 'Aguardar liberação + inserir promessa no portal'
vendas_df.loc[(vendas_df['Faturar a partir de'] <= today) & (vendas_df['Promise Date'] == ''), 'Ação']= 'Faturar pedido + inserir promessa'

#exp de estilização =>  df.style.applymap(lambda x: 'background-color : yellow' if x>df.iloc[0,0] else '')
vendas_df.style.applymap(lambda x: 'background-color : green' if x=='Faturar pedido' else '')

occur=vendas_df.groupby(["Ação"]).size()
display(occur)

vendas_df.rename(columns = {'Need By Date - Atual':'Data de Necessidade da PO'}, inplace = True)
vendas_df.rename(columns = {'New Need By Date (PO)':'Data de Necessidade atualizada da produção'}, inplace = True)

writer = pd.ExcelWriter('teste.xlsx',engine=None ,date_format='dd/mmm/yyyy')


vendas_df.to_excel(writer,index=False,sheet_name='teste',startrow=1,startcol =0, header= False)
workbook = writer.book
worksheet = writer.sheets['teste']
worksheet.set_default_row(20)
worksheet.set_row(0, 25)
worksheet.autofilter



cell_format = workbook.add_format()
cell_format.set_font_size(8)
cell_format.set_align('center')
(a, b) = vendas_df.shape



worksheet.set_column(0, b-1,15,cell_format)



header_format = workbook.add_format({
'bold': False,
'text_wrap': False,
'valign': 'center',
'align':'center',
'fg_color' : 'F0F4FA',
'border': 1,
'font_name': 'Calibri',
'font_size': 8
})



for col_num, value in enumerate(vendas_df.columns.values):
    worksheet.write(0, col_num, value, header_format)




# Close the Pandas Excel writer and output the Excel file.
worksheet.autofilter(0,0,0,b-1)



writer.save()

#ajustando colunas
from openpyxl.styles import Font
import openpyxl
    
wb = openpyxl.load_workbook(r"C:\Users\matheus.arruda\Desktop\Matheus\teste.xlsx")
sheet = wb.active
sheet.column_dimensions['C'].width = 13
sheet.column_dimensions['D'].width = 26
sheet.column_dimensions['G'].width = 55
sheet.column_dimensions['Q'].width = 35
wb.save("Teste.xlsx")

#teste pintando com coluna REF             # funcionando seeem 
from openpyxl import load_workbook, Workbook
from openpyxl.styles import PatternFill,Font,colors, Alignment
from openpyxl.formatting.rule import CellIsRule
from openpyxl.formatting.rule import Rule

planilha = load_workbook(r"C:\Users\matheus.arruda\Desktop\Matheus\teste.xlsx")
aba_ativa = planilha.active

for celula in aba_ativa["Q"]:
    if (celula.value == "Faturar pedido"):
        fill_pattern= PatternFill(patternType='solid', 
                              fgColor='8ee53f')
        linha = celula.row
        aba_ativa[f"Q{linha}"].fill=fill_pattern

for celula in aba_ativa["Q"]:
    if (celula.value == "Faturar pedido + inserir promessa"):
        fill_pattern= PatternFill(patternType='solid', 
                              fgColor='fdee00')
        linha = celula.row
        aba_ativa[f"Q{linha}"].fill=fill_pattern
for celula in aba_ativa["Q"]:
    if (celula.value == "Aguardar liberação + inserir promessa no portal"):
        fill_pattern= PatternFill(patternType='solid', 
                              fgColor='d63737') #VERM VIVO
        linha = celula.row
        aba_ativa[f"Q{linha}"].fill=fill_pattern    

for celula in aba_ativa["P"]:
    if (celula.value == "Past Due Promise"):
        fill_pattern= PatternFill(patternType='solid', 
                              fgColor='fcc0b8') #red claro
        linha = celula.row
        aba_ativa[f"P{linha}"].fill=fill_pattern    
for celula in aba_ativa["P"]:
    if (celula.value == "Bad Promise"):
        fill_pattern= PatternFill(patternType='solid', 
                              fgColor='ffe0db') #red mais claro
        linha = celula.row
        aba_ativa[f"P{linha}"].fill=fill_pattern  
for celula in aba_ativa["P"]:
    if (celula.value == "No Promise"):
        fill_pattern= PatternFill(patternType='solid', 
                              fgColor='d63737') #VERM VIVO
        linha = celula.row
        aba_ativa[f"P{linha}"].fill=fill_pattern     
for celula in aba_ativa["P"]:
    if (celula.value == "Promise OK"):
        fill_pattern= PatternFill(patternType='solid', 
                              fgColor='8ee53f') #GREEN
        linha = celula.row
        aba_ativa[f"P{linha}"].fill=fill_pattern   
        
   
        #redfill=PatternFill(start_color='EE4636',end_color='EE4636', fill_type='solid')
        #sh.conditional_formatting.add('z',CellIsRule(operator='equal',formula=['1'],stopIfTrue=True,fill=redfill))
planilha.save("Teste.xlsx")

#Envia por email arquivo

import win32com.client as win32
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'matheus.arruda@Wabtec.com'
mail.Subject = 'OPEN PO FUP'
mail.Body = "Caro fornecedor,\nSegue em anexo a nossa carteira atualizada.\n\nFavor validar as informações, seguir com faturamentos autorizados e ações indicadas na planilha em anexo. \n\n"+str(occur.to_string())+"\n\nChecar Status da coluna ‘Promise Status’ e atuar se necessário.\n\nAtenciosamente,\nMatheus Arruda \nFulfillment "

attachment  = (r"C:\Users\matheus.arruda\Desktop\Matheus\teste.xlsx")
mail.Attachments.Add(attachment)

mail.Send()

(16824, 68)
(5957, 68)
(5929, 74)


Ação
Aguardar liberação + inserir promessa no portal     289
Aguardar liberação, não autorizado                 2549
Faturar pedido                                     3058
Faturar pedido + inserir promessa                    33
dtype: int64